In [1]:
import gym
import gym_coverage
import random
import numpy as np
import tensorflow as tf

from CNNDDPGAgent import CNNDDPGAgent
from collections import deque

from matplotlib import pyplot as plt

seed = 0
np.random.seed(seed)
random.seed(seed)

In [2]:
env = gym.make('CoverageCar-v0')
env.seed(seed)
N_F_pos = env.observation_space.spaces[0].shape[0]
N_F_map = env.observation_space.spaces[1].shape[0]
#* env.observation_space.spaces[1].shape[1]
N_F = N_F_pos + N_F_map
print(env.action_space)
N_A = env.action_space.shape[0]
agent = CNNDDPGAgent(N_F_pos, N_F_map, N_A, epsilon_decay= 0.99999, memory_mode='PER', batch_size=64)

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
Box(2,)


In [3]:
import time
max_t = 100
avg_return_list = deque(maxlen=10)
avg_loss_list = deque(maxlen=10)
for i in range(500000):
    obs = env.reset()
    obs = np.concatenate((obs[0], obs[1].flatten()))
    done = False
    total_reward = 0
    total_loss = 0
    #env.render()
    #time.sleep(0.5)
    for t in range(max_t):
        action = agent.get_action(np.reshape(obs, (1, -1)))
        next_obs, reward, done, info = env.step(action[0])
        next_obs = np.concatenate((next_obs[0], next_obs[1].flatten()))
        agent.add_experience(obs, action[0], reward, next_obs, done)
        
        loss = agent.train_model()
        agent.update_memory(t, max_t)
        agent.update_policy()
        #env.render()
        #time.sleep(0.5)
        obs = next_obs
        total_reward += reward
        total_loss += loss
        if done:
            break
            
    agent.update_target()
    avg_return_list.append(total_reward)
    avg_loss_list.append(total_loss)
    
    if (i%1000)==0:
        print('{} loss : {:.3f}, return : {:.3f}, eps : {:.3f}'.format(i, np.mean(avg_loss_list), np.mean(avg_return_list), agent.epsilon))
#agent.save_model(path="model/coverage_6_model_new.ckpt")
#agent.restore_model(path="model/coverage_6_model.ckpt")

0 loss : nan, return : -100.000, eps : 0.999
1000 loss : 0.006, return : -100.000, eps : 0.368


KeyboardInterrupt: 